# Исследование надежности заемщиков


Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


# Описание проекта.

По представленным статичтическим данным о платежеспособности клиентов банка необходимо провести исследование- влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.  
Цель исследования — проверить гипотезы:
Существует ли зависимость:

-между наличием детей и возвратом кредита в срок;  
-между семейным положением и возвратом кредита в срок;  
-между уровнем дохода и возвратом кредита в срок;   
-между целью кредита и его возврат в срок.  
Ход исследования:  
Данные о поведении пользователей (статистика о платёжеспособности клиентов)- в файле /datasets/data.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.  

Проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки поищем возможность исправить самые критичные ошибки данных.  

Таким образом, исследование пройдёт в три этапа:  

Обзор данных.  
Предобработка данных.  
Проверка гипотез.

## Обзор данных

#### Импортируем библиотеку pandas. Считываем данные из csv-файла в датафрейм и сохраняем в переменную `data`. 

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

#### Выведем первые 5 строчек датафрейма `data` на экран.

In [2]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Выведем основную информацию о датафрейме с помощью метода `info()`.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

#### Выведем количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

####  Заполним пропуски медианным значением по каждому типу из столбца `income_type`.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

#### Заменим все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

#### Выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

#### Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

#### В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

#### Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

#### Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

#### Выведем количество пропущенных значений для каждого столбца с помощью двух методов. 

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

#### Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

#### Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

#### Приведем к нижнему регистру. 

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

#### Создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.




In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

#### Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

#### Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [22]:
chilldren_debt=data.groupby('children').agg({'debt':['count','sum']}) # группировка по колонке 'chilldren'  и подсчет количества/суммы значений в колонке 'debt' с помощью функции agg
proportion=chilldren_debt['debt']['sum']/chilldren_debt['debt']['count'] #расчет соотношения суммы к количеству по результатам гуппировки
print(proportion)

children
0    0.075353
1    0.092327
2    0.094542
3    0.081818
4    0.097561
5    0.000000
dtype: float64


**Вывод:** Мы видим, что возврат кредита зависит от количества детей в семье: долга нет в семье с 5 детьми, наименьший долг в семье без ребенка, наибольшй долг в семьях с 1,2,3,4 детьми.

**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [23]:
status_debt=data.groupby('family_status').agg({'debt':['count','sum']}) # группировка по колонке 'family_status'  и подсчет количества/суммы значений в колонке 'debt' с помощью функции agg
proportion=status_debt['debt']['sum']/status_debt['debt']['count'] #расчет соотношения суммы к количеству по результатам гуппировки
print(proportion)

family_status
Не женат / не замужем    0.097639
в разводе                0.070648
вдовец / вдова           0.066246
гражданский брак         0.092861
женат / замужем          0.075575
dtype: float64


**Вывод:** Зависимость между сеейным положением и возвратом кредита в срок существует: самый худший показатель у людей в статусе"не женат/не замужем" и "гражданский брак", средний показатель- в статусе "в разводе" и "женат/замужем", самый хороший - в статусе "вдовец/вдова"

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [24]:
income_debt=data.groupby('total_income_category').agg({'debt':['count','sum']}) # группировка по колонке 'total_income_category'  и подсчет количества/суммы значений в колонке 'debt' с помощью функции agg
proportion=income_debt['debt']['sum']/income_debt['debt']['count'] #расчет отношения суммы к количеству по результатам гуппировки
print(proportion)

total_income_category
A    0.080000
B    0.070602
C    0.084891
D    0.060172
E    0.090909
dtype: float64


Ранее в датафрейме data мы создавали столбец total_income_category с категориями:
0–30000 — 'E';
30001–50000 — 'D';
50001–200000 — 'C';
200001–1000000 — 'B';
1000000 и выше — 'A'
Зависимость между уровнем дохода и возвратом кредита в срок есть. Мы видим, что самый плохой показатель у людей с наименьшим доходом, самый хороший - с уровнем дохода от 30 001 до 50 000, у людей с доходом от 50 001 и выше - средний показатель.



**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

In [25]:
purpose_debt=data.groupby('purpose_category').agg({'debt':['count','sum']}) # группировка по колонке 'purpose_category'  и подсчет количества/суммы значений в колонке 'debt' с помощью функции agg
proportion=purpose_debt['debt']['sum']/purpose_debt['debt']['count'] #расчет отношения суммы к количеству по результатам гуппировки (по возрастанию)
print(proportion)

purpose_category
операции с автомобилем      0.093436
операции с недвижимостью    0.072531
получение образования       0.092504
проведение свадьбы          0.078744
dtype: float64


**Вывод:** Самый большой показатель по долгам у людей с целью "операции с автомобилем" и "получение образования", самый маленький - "операции с недвижимостью" и "проведение свадьбы"


**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*Ответ:*Ошибки на этапе заполнения первичной иформации, ошибки при загрузке файла. 

**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Ответ:* Когда некоторые значения сильно выделяются среди большинства,то среднее значение некорректно характеризует данные. Поэтому заполнение пропусков медианным значением - это более объективное решение.

## Общий вывод.

**Вывод**  
В результате моих данных, можно сделать вывод: 
-самыми ненадежными заемщиками являются клиенты с маленьким доходом (до 30 т.р.), не женатые/не замужние с целью кредита на покупку автомобиля;
-наименьшую задолженность имеют люди с доходом от 30 т.р. до 50т.р. (почти такой же показатель у людей с доходом от 200 т.р. до 1000 т.р.), в статусе вдова/вдовец либо в разводе, не имеющие детей и с целью кредита на покупку жилья или проведение свадьбы.


